In [1]:
import dace
import random
import cupy as cp

from dace import registry
from dace.sdfg.scope import ScopeSubgraphView
from dace.codegen.prettycode import CodeIOStream
from dace.codegen.targets.target import TargetCodeGenerator
from dace.codegen.targets.framecode import DaCeCodeGenerator
from dace.codegen.targets.cpp import sym2cpp
from IPython.display import Code
from dace.config import Config


print(Config.get('compiler', 'cuda', 'implementation'))

experimental


In [2]:
@dace.program
def warpLevel(A: dace.float64[512] @ dace.dtypes.StorageType.GPU_Global, B: dace.float64[512] @ dace.dtypes.StorageType.GPU_Global):
    for i in dace.map[0:512:512] @ dace.dtypes.ScheduleType.GPU_Device:
        for j in dace.map[0:512] @ dace.dtypes.ScheduleType.GPU_ThreadBlock:
            for wi in dace.map[0:16] @ dace.dtypes.ScheduleType.GPU_Warp:
                A[wi] = A[wi] + 1

sdfg = warpLevel.to_sdfg()
sdfg

SDFG (warpLevel)

In [3]:
Code(sdfg.generate_code()[1].clean_code, language='cpp')

#include <cuda_runtime.h>
#include <dace/dace.h>


struct warpLevel_state_t {
    dace::cuda::Context *gpu_context;
};



DACE_EXPORTED int __dace_init_experimental_cuda(warpLevel_state_t *__state);
DACE_EXPORTED int __dace_exit_experimental_cuda(warpLevel_state_t *__state);



int __dace_init_experimental_cuda(warpLevel_state_t *__state) {
    int count;

    // Check that we are able to run cuda code
    if (cudaGetDeviceCount(&count) != cudaSuccess)
    {
        printf("ERROR: GPU drivers are not configured or cuda-capable device "
               "not found\n");
        return 1;
    }
    if (count == 0)
    {
        printf("ERROR: No cuda-capable devices found\n");
        return 2;
    }

    // Initialize cuda before we run the application
    float *dev_X;
    DACE_GPU_CHECK(cudaMalloc((void **) &dev_X, 1));
    DACE_GPU_CHECK(cudaFree(dev_X));

    

    __state->gpu_context = new dace::cuda::Context(1, 1);

    // Create cuda streams and events
    for(int i = 0; i < 1; ++i) {
        DACE_GPU_CHECK(cudaStreamCreateWithFlags(&__state->gpu_context->internal_streams[i], cudaStreamNonBlocking));
        __state->gpu_context->streams[i] = __state->gpu_context->internal_streams[i]; // Allow for externals to modify streams
    }
    for(int i = 0; i < 1; ++i) {
        DACE_GPU_CHECK(cudaEventCreateWithFlags(&__state->gpu_context->events[i], cudaEventDisableTiming));
    }

    

    return 0;
}

int __dace_exit_experimental_cuda(warpLevel_state_t *__state) {
    

    // Synchronize and check for CUDA errors
    int __err = static_cast<int>(__state->gpu_context->lasterror);
    if (__err == 0)
        __err = static_cast<int>(cudaDeviceSynchronize());

    // Destroy cuda streams and events
    for(int i = 0; i < 1; ++i) {
        DACE_GPU_CHECK(cudaStreamDestroy(__state->gpu_context->internal_streams[i]));
    }
    for(int i = 0; i < 1; ++i) {
        DACE_GPU_CHECK(cudaEventDestroy(__state->gpu_context->events[i]));
    }

    delete __state->gpu_context;
    return __err;
}

DACE_EXPORTED bool __dace_gpu_set_stream(warpLevel_state_t *__state, int streamid, gpuStream_t stream)
{
    if (streamid < 0 || streamid >= 1)
        return false;

    __state->gpu_context->streams[streamid] = stream;

    return true;
}

DACE_EXPORTED void __dace_gpu_set_all_streams(warpLevel_state_t *__state, gpuStream_t stream)
{
    for (int i = 0; i < 1; ++i)
        __state->gpu_context->streams[i] = stream;
}

__global__ void warpLevel_3_0_0_0(double * __restrict__ A)
{ // Kernel scope (open 1)
    int i = (512 * blockIdx.x);
    { // ThreadBlock Scope (open 1)
        int j = threadIdx.x;
        { // WarpLevel Scope (open 1)
            double __tmp3;

            int warpId = (threadIdx.x + blockDim.x * threadIdx.y + blockDim.x * blockDim.y * threadIdx.z) / 32;
            if ( 0 < warpId && warpId < 15) { // WarpLevel Scope (open 2)



                ----------------------------------
                // WarpLevel operations here
                ----------------------------------



            } // WarpLevel Scope (close 1)
        } // WarpLevel Scope (close 2)
    } // ThreadBlock Scope (close 1)
} // Kernel scope (close 1)


DACE_EXPORTED void __dace_runkernel_warpLevel_3_0_0_0(warpLevel_state_t *__state, double * __restrict__ A);
void __dace_runkernel_warpLevel_3_0_0_0(warpLevel_state_t *__state, double * __restrict__ A)
{


    void  *warpLevel_3_0_0_0_args[] = { (void *)&A };
    gpuError_t __err = cudaLaunchKernel( (void*)warpLevel_3_0_0_0, dim3(1, 1, 1), dim3(512, 1, 1), warpLevel_3_0_0_0_args, 0, __state->gpu_context->streams[0]
    );

    DACE_KERNEL_LAUNCH_CHECK(__err, "warpLevel_3_0_0_0", 1, 1, 1, 512, 1, 1);
}